Week 5, Assignment C1:

Logistic Regression with Yahoo! Finance API in Python

In this notebook, we will study Logistic Regression using the Yahoo! Finance API in Python.

Please complete the lines bellow where you see "#TODO".

In [1]:
#Objective 1.
#Use this cell to import the Numpy (as np), Pandas (as pd), and YFinance (as yf) packages.

import numpy as np
import pandas as pd
import pandas
import yfinance as yf


In [34]:
#Our work will also require some components of the Sklearn and Pandas_Datareader packages as imported below:

import sklearn
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from pandas_datareader import data as pdr
yf.pdr_override()

import warnings
warnings.filterwarnings("ignore")


In [35]:
#Objective 2.
#Select a stock symbol for a stock whose historical data is available on the Yahoo! Finance website.  Store the
#string of your chosen symbol to the new variable "stock_symbol".

stock_symbol = "BTC-USD"

In [36]:
#Objective 3.
#Use the function "pdr.get_data_yahoo(stock_symbol, start_date, end_date)" to generate a Pandas dataframe of
#historical stock data for your chosen stock.  Retain only the first four columns of the dataframe.  Be sure to 
#drop any rows containing NaN's, and take a peek at the resulting dataframe to make sure everything looks good.

df = pdr.get_data_yahoo(stock_symbol, pandas.Timestamp('2019'), pandas.Timestamp('2021'))
df = df[df.columns[0:4]]
df = df.dropna()
df

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close
Date,,,,
2018-12-31,3866.839111,3868.742920,3725.867432,3742.700439
2019-01-01,3746.713379,3850.913818,3707.231201,3843.520020
2019-01-02,3849.216309,3947.981201,3817.409424,3943.409424
2019-01-03,3931.048584,3935.685059,3826.222900,3836.741211
2019-01-04,3832.040039,3865.934570,3783.853760,3857.717529
...,...,...,...,...
2020-12-27,26439.373047,28288.839844,25922.769531,26272.294922
2020-12-28,26280.822266,27389.111328,26207.640625,27084.808594
2020-12-29,27081.810547,27370.720703,25987.298828,27362.437500


In [37]:
#Objective 4.
#Use the historical data to define some predictor variables.  Add these variables to the dataframe.  Include, at a
#minimum, (Predictor.I) the rolling average closing price over the past fifteen (15) days and (Predictor.II) the
#change in opening price over the past one (1) day.  Be sure to again drop any rows containing NaN's, and take a
#peek at the resulting dataframe to make sure everything looks good.  Store the dataframe as the new variable "X".

df['Close_15_Rolling'] = df['Close'].rolling(window=15).mean()
df['Open_1_Change'] = df['Open'].diff()
df = df.dropna()
X = df
df

,Open,High,Low,Close,Close_15_Rolling,Open_1_Change
Date,,,,,,
2019-01-14,3557.311035,3727.836182,3552.285156,3706.052246,3834.926986,-101.557129
2019-01-15,3704.216309,3720.153320,3619.949219,3630.675293,3827.458643,146.905273
2019-01-16,3631.509766,3685.777100,3624.673340,3655.006836,3814.891097,-72.706543
2019-01-17,3651.871094,3680.135986,3621.960938,3678.563965,3797.234733,20.361328
2019-01-18,3677.990479,3682.520020,3637.080811,3657.839355,3785.307943,26.119385
...,...,...,...,...,...,...
2020-12-27,26439.373047,28288.839844,25922.769531,26272.294922,22889.632031,1762.357422
2020-12-28,26280.822266,27389.111328,26207.640625,27084.808594,23419.127083,-158.550781
2020-12-29,27081.810547,27370.720703,25987.298828,27362.437500,23960.179948,800.988281


In [39]:
#Objective 5.
#Define the target or dependent variable to be one (1) if the change in closing price over the past one (1) day is
#nonnegative and negative one (-1) if the change in closing price over the past one (1) day is negative.  This
#variable's values should be forward-looking (i.e., you should subtract today's price from tomorrow's price rather
#than subtracting yesterday's price from today's price).  Store the resulting values as the new variable "y".  You
#may find the "np.where(*args)" function to be useful.  Look it up in Numpy documentation for support.

y = np.where((df.Close.shift(-1)-df['Close'])>=0,1,-1)


In [40]:
#Objective 6.
#Split the data into training and test sets, putting the first seventy percent (70%) of the data in the training
#set.

index = int(0.7*len(X))
X_train = X[:index]
X_test = X[index:]
y_train = y[:index]
y_test = y[index:]

In [41]:
#Objective 7.
#Instantiate the Logistic Regression model object, and use its ".fit(*args)" method to fit the model to the 
#training data.

logistic = LogisticRegression()
logistic = logistic.fit(X_train, y_train)

In [9]:
#Objective 8.
#Examine the model's coefficients by using its ".coef_" method.

pd.DataFrame({'column':X.columns.values, 'b': logistic.coef_.ravel()})

,column,b
0,Open,-0.001610
1,High,0.002032
2,Low,0.001717
3,Close,-0.002222
4,Close_15_Rolling,0.000085
5,Open_1_Change,0.000023


In [42]:
#Objective 9.
#Use the model's ".predict_proba(*args)" and ".predict(*args)" methods to generate predictions over the test set.

probabilities = logistic.predict_proba(X_test)
predictions = logistic.predict(X_test)

In [11]:
#Objective 10.
#Use the function "metrics.confusion_matrix(*args)" to create a confusion matrix comparing the predicted and true
#classification labels over the test set.

sklearn.metrics.confusion_matrix(y_pred=predictions, y_true=y_test)


array([[62, 28],
       [72, 53]], dtype=int64)

In [12]:
#Objective 11.
#Calculate the model's accuracy on the test set using its ".score(*args)" method.

logistic.score(X_test, y_test)

0.5348837209302325

In [13]:
#Objective 12.
#Use five-fold cross validation to cross-check the accuracy of the model over different held-out test sets.  This
#is where you should use the function "cross_val_score(*args)".

cross_val = cross_val_score(LogisticRegression(), X=X, y=y, cv=5)
cross_val

array([0.57342657, 0.51048951, 0.52447552, 0.55244755, 0.52112676])